In [ ]:
#connect to google drive
!pip install gshell
!gshell init

# Setup
**Make sure to read the instructions carefully!**

If you have other resources used in the Blender project and chose to *make all paths relative*, pack all of them into a zip archive. Alternatively, you can *pack all external file*.

* `blender_version` : Version of blender used to render the scene.
* `blend_file_path` : Path to the blend file after unpacking the zip archive. If blend file is used, this is automatically ignored.
___
* `upload_type` : Select the type of upload method. `gdrive_relative` pulls everything from the folder specified.
* `drive_path` : Path to your blend/zip file relative to the root of your Google Drive if `google_drive` is selected. Must  state the file and its extension (.zip/.blend) **unless** `gdrive_relative` is selected.
* `url_blend` : Specify the URL to the blend/zip file if `url` is selected.
___
* `animation` : Specify whether animation or still image is rendered. If **still image** is used, put the frame number in `start_frame`.
* `start_frame, end_frame` : Specify the start and end frame for animation. You may put same value such as zero for both input to set the default frame in the blend file.
___
* `download_type` : Select the type of download method. `gdrive_direct` enables the frames to be outputted directly to Google Drive (zipping will be disabled).
* `output_name` : Name of the output frames, **do NOT include .blend!** (## for frame number)
* `zip_files` : Archive multiple animation frames automatically into a zip file.
* `drive_output_path` : Path to your frames/zip file in Google Drive.
___
* `gpu_enabled, cpu_enabled` : Toggle GPU and CPU for rendering. CPU might give a slight boost in rendering time but may varies depend on the project.

After you are done, go to Runtime > Run All (Ctrl + F9) and upload your files or have Google Drive authorised below. See the [GitHub repo](https://github.com/syn73/blender-colab) for more info.

In [ ]:
blender_version = '3.0.1' #@param ['2.79b', '2.80rc3', '2.81a', '2.82a', '2.83.18', '2.90.1', '2.91.2', '2.92.0', '2.93.6', '3.0.0', '3.0.1'] {allow-input: false}
# from google drive folder url https://drive.google.com/file/d/14rU-w4UwEkRoslf6zGqFceu5ade7AU_O/view?usp=sharing 
# use 14rU-w4UwEkRoslf6zGqFceu5ade7AU_O as id
blend_file_id = '14rU-w4UwEkRoslf6zGqFceu5ade7AU_O' #@param {type: 'string'}
start_frame =  2#@param {type: 'integer'}
end_frame =  4#@param {type: 'integer'}
# from google drive folder url https://drive.google.com/drive/u/0/folders/1ph2v7i3Qm8U6KVPh1DEJ1UCXRKVMYEY0  
# use 1ph2v7i3Qm8U6KVPh1DEJ1UCXRKVMYEY0 as id
output_directory_id = '1ph2v7i3Qm8U6KVPh1DEJ1UCXRKVMYEY0' #@param ['xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'] {allow-input: true}
#@markdown ---
gpu_enabled = False #@param {type:"boolean"}
cpu_enabled = True #@param {type:"boolean"}
#@markdown ---
blend_file_path = '/content/particles.blend' #@param {type: 'string'}
temp_output_path = '/content/out' #@param {type: 'string'}
output_name = 'blender-####' #@param {type: 'string'}


In [ ]:
import os
blender_url_dict = {'2.79b'   : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.79/blender-2.79b-linux-glibc219-x86_64.tar.bz2",
                    '2.80rc3' : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.80/blender-2.80rc3-linux-glibc217-x86_64.tar.bz2",
                    '2.81a'   : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.81/blender-2.81a-linux-glibc217-x86_64.tar.bz2",
                    '2.82a'   : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.82/blender-2.82a-linux64.tar.xz",
                    '2.83.18' : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.83/blender-2.83.18-linux-x64.tar.xz",
                    '2.90.1'  : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.90/blender-2.90.1-linux64.tar.xz",
                    '2.91.2'  : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.91/blender-2.91.2-linux64.tar.xz",
                    '2.92.0'  : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.92/blender-2.92.0-linux64.tar.xz",
                    '2.93.6'  : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.93/blender-2.93.6-linux-x64.tar.xz",
                    '3.0.0'   : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender3.0/blender-3.0.0-linux-x64.tar.xz",
                    '3.0.1'   : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender3.0/blender-3.0.1-linux-x64.tar.xz"}

blender_url = blender_url_dict[blender_version]
base_url = os.path.basename(blender_url)

!mkdir $blender_version
!wget -nc $blender_url
!tar -xkf $base_url -C ./$blender_version --strip-components=1

In [ ]:
# Enable GPU rendering (or add custom properties here)
data = "import re\n"+\
    "import bpy\n"+\
    "scene = bpy.context.scene\n"+\
    "scene.cycles.device = 'GPU'\n"+\
    "prefs = bpy.context.preferences\n"+\
    "prefs.addons['cycles'].preferences.get_devices()\n"+\
    "cprefs = prefs.addons['cycles'].preferences\n"+\
    "print(cprefs)\n"+\
    "for compute_device_type in ('CUDA', 'OPENCL', 'NONE'):\n"+\
    "    try:\n"+\
    "        cprefs.compute_device_type = compute_device_type\n"+\
    "        print('Device found:',compute_device_type)\n"+\
    "        break\n"+\
    "    except TypeError:\n"+\
    "        pass\n"+\
    "for device in cprefs.devices:\n"+\
    "    if not re.match('intel', device.name, re.I):\n"+\
    "        print('Activating',device)\n"+\
    "        device.use = "+str(gpu_enabled)+"\n"+\
    "    else:\n"+\
    "        device.use = "+str(cpu_enabled)+"\n"
with open('setgpu.py', 'w') as f:
    f.write(data)

In [ ]:
!nvidia-smi

```
#set working directory
!gshell cd --with-id '{output_directory_id}'
#make temp output directory
!mkdir -p '{temp_output_path}'
#blend file downloaad from google drive
!gshell download -i '{blend_file_id}'

if not temp_output_path.endswith('/'):
    temp_output_path += '/'
# concatenating /content/out + blender-#### >>  /content/out/blender-#### to be used in blender output
output_path = temp_output_path + output_name
#as if frames 1 to frame 250 or frame 400 or frame xxx
framelist = range(start_frame, end_frame+1)
#initiating as previously processed files in directory as none (not exactly) as  later while using "gshell ls" would put out list of files as ['a.txt','b.txt',...] then doing len(files) as finding length would give no of files, which is to keep count of files processed and uploaded
nooffiles=''

#for generating each frame and uploading them frames are to be formatted as blender-0001.png, blender-0002.png, ...
for index in framelist:  
  nooffiles = !gshell ls
  index = len(nooffiles) + index
  !sudo ./$blender_version/blender -b 'render/{blend_file_path}' -P setgpu.py -E CYCLES -o '{output_path}' -noaudio -f $index
  #uploading to destination from source
  !gshell upload --parent '{output_directory_id}' --recursive '{temp_output_path}'/*
  #emptying source (temporary directory)
  !rm -r '{temp_output_path}'/*
```


In [ ]:
!gshell cd --with-id '{output_directory_id}'
!mkdir -p '{temp_output_path}'
!gshell download -i '{blend_file_id}'

if not temp_output_path.endswith('/'):
    temp_output_path += '/'
    
output_path = temp_output_path + output_name

framelist = range(start_frame, end_frame+1)
nooffiles=''

for index in framelist:  
  nooffiles = !gshell ls
  index = len(nooffiles) + index
  !sudo ./$blender_version/blender -b '{blend_file_path}' -P setgpu.py -E CYCLES -o '{output_path}' -noaudio -f $index
  !gshell upload --parent '{output_directory_id}' --recursive '{temp_output_path}'/*
  !rm -r '{temp_output_path}'/*

# ***EXIT SECTION***

## Disclaimer
The GPU used in Google Colab is specialized for data centres, neural network etc, not rendering 3D scenes. Because the computing power provided are free, the usage limits, idle timeouts and speed of the rendering may varies. [ColabPro](https://colab.research.google.com/signup) is available for those who wanted to have more powerful GPU and longer session for rendering. See the [FAQ](https://research.google.com/colaboratory/faq.html) for more info about this platform. In some cases, it might be faster to use an online Blender renderfarm.

## License
```
MIT License

Copyright (c) 2020 syn73

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
```